In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from config.config import ITEM_URL_BASE, BASE_URL, HOME_FILE, OUTPUT_FILE
from logger.logger import logger  # Import the configured logger

headers = {'User-Agent': 'Mozilla/5.0'}

def get_unique_item_ids(max_pages=300):
    """Fetch unique item IDs from paginated pages with proper logging."""
    item_ids = set()
    page = 1
    has_more_pages = True
    
    logger.info(f"Starting to scrape item IDs (max pages: {max_pages})")
    
    while has_more_pages:
        if max_pages and page > max_pages:
            logger.info(f"Reached max pages limit ({max_pages})")
            break
            
        url = f"{BASE_URL}?page={page}"
        logger.debug(f"Fetching page {page}: {url}")
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                logger.error(f"Failed to fetch page {page}. Status code: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find all item links and extract IDs from hrefs
            current_page_ids = set()
            for a in soup.select("a.item_link"):
                href = a.get("href")
                if href:
                    match = re.search(r'/items/(\d+)', href)
                    if match:
                        item_id = match.group(1)
                        current_page_ids.add(item_id)
            
            # Check if we found any new items on this page
            if not current_page_ids:
                logger.info(f"No items found on page {page}, stopping pagination")
                has_more_pages = False
            else:
                new_items = current_page_ids - item_ids
                if not new_items:
                    logger.info(f"No new items found on page {page}, stopping pagination")
                    has_more_pages = False
                else:
                    logger.info(f"Found {len(new_items)} new items on page {page}")
                    item_ids.update(current_page_ids)
                    page += 1
                    
        except requests.RequestException as e:
            logger.error(f"Network error fetching page {page}: {str(e)}", exc_info=True)
            break
        except Exception as e:
            logger.error(f"Unexpected error processing page {page}: {str(e)}", exc_info=True)
            break

    # Save raw HTML for debugging
    try:
        with open(HOME_FILE, mode="w", newline="", encoding="utf-8") as f:
            f.write(response.text)
        logger.debug(f"Saved last page HTML to {HOME_FILE}")
    except Exception as e:
        logger.error(f"Failed to save HTML to {HOME_FILE}: {str(e)}", exc_info=True)

    logger.info(f"Finished scraping. Found {len(item_ids)} unique items")
    return item_ids

# Run the function and print results
item_ids = get_unique_item_ids()
links = list(item_ids)
print(links)
print(len(links))

2025-05-11 23:31:13 - scraper - INFO - Starting to scrape item IDs (max pages: 300)
2025-05-11 23:31:13 - scraper - INFO - Found 28 new items on page 1
2025-05-11 23:31:14 - scraper - INFO - Found 28 new items on page 2
2025-05-11 23:31:15 - scraper - INFO - Found 28 new items on page 3
2025-05-11 23:31:16 - scraper - INFO - Found 28 new items on page 4
2025-05-11 23:31:17 - scraper - INFO - Found 28 new items on page 5
2025-05-11 23:31:17 - scraper - INFO - Found 28 new items on page 6
2025-05-11 23:31:18 - scraper - INFO - Found 28 new items on page 7
2025-05-11 23:31:19 - scraper - INFO - Found 28 new items on page 8
2025-05-11 23:31:20 - scraper - INFO - Found 28 new items on page 9
2025-05-11 23:31:20 - scraper - INFO - Found 26 new items on page 10
2025-05-11 23:31:21 - scraper - INFO - Found 28 new items on page 11
2025-05-11 23:31:22 - scraper - INFO - Found 28 new items on page 12
2025-05-11 23:31:23 - scraper - INFO - Found 28 new items on page 13
2025-05-11 23:31:24 - scrape

['5165411', '5150136', '5117823', '5193713', '5193372', '4286568', '5194103', '5193856', '4940552', '5120026', '5154942', '5144377', '5138864', '4542167', '5194151', '5185406', '5122950', '4876404', '5179990', '5153860', '5121336', '4596994', '4509504', '5116034', '5173186', '5123130', '5129327', '5192789', '5163010', '4875516', '4303498', '5127671', '5193167', '5190647', '5176307', '5150735', '5193627', '5120337', '5193992', '5193947', '4103366', '5079727', '5075150', '5180594', '5058488', '5109481', '4523256', '5192975', '5179500', '5175449', '5193706', '5192827', '5042401', '3853333', '5056614', '5118265', '5058889', '5191312', '5192725', '5105601', '5169602', '5168004', '5176951', '5096475', '5192356', '5163938', '4487889', '5173481', '5163785', '5193397', '4886113', '4927809', '5193236', '5099037', '5120413', '5190410', '5184847', '5182565', '5189523', '5163359', '5173221', '4765292', '4632009', '5194020', '5109667', '5134107', '5182456', '4915365', '5143948', '5114350', '5177134'

In [2]:
def extract_item_data(link):
    """Extract detailed item data from individual product page with comprehensive logging"""
    url = ITEM_URL_BASE + link
    logger.debug(f"Starting to extract data from: {url}")
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Failed to fetch {url} - Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract item ID
        item_id = link.split("/")[-1]
        logger.debug(f"Processing item ID: {item_id}")

        # Extract amount and currency using meta tags
        amount_tag = soup.find('meta', {'property': 'product:price:amount'})
        currency_tag = soup.find('meta', {'property': 'product:price:currency'})

        amount = amount_tag['content'] if amount_tag else None
        currency = currency_tag['content'] if currency_tag else None
        logger.debug(f"Extracted price: {amount} {currency}")

        # Extract property fields dynamically
        properties = {}
        prop_div = soup.find("div", class_="product-properties__column")
        if prop_div:
            for prop in prop_div.find_all("div", class_="product-properties__i"):
                try:
                    name = prop.find("label", class_="product-properties__i-name").get_text(strip=True)
                    value = prop.find("span", class_="product-properties__i-value").get_text(strip=True)
                    properties[name] = value
                    logger.debug(f"Found property: {name} = {value}")
                except Exception as e:
                    logger.warning(f"Failed to extract property: {str(e)}", exc_info=True)

        # Extract locations
        location_tags = soup.find_all('a', {'data-stat': 'product-locations'})
        locations = [tag.get_text(strip=True) for tag in location_tags] if location_tags else []
        logger.debug(f"Found locations: {locations}")

        # Construct result dictionary
        item_data = {
            'item_id': item_id,
            'amount': amount,
            'currency': currency,
            'location': locations
        }

        # Add all found properties dynamically
        item_data.update(properties)
        
        logger.info(f"Successfully processed item: {item_id}")
        return item_data

    except requests.RequestException as e:
        logger.error(f"Network error while processing {url}: {str(e)}", exc_info=True)
        return None
    except Exception as e:
        logger.error(f"Unexpected error processing {url}: {str(e)}", exc_info=True)
        return None


def scrape_all_items():
    """Main scraping function that processes all items with progress tracking"""
    if not links:
        logger.warning("No links to process - empty list provided")
        return

    results = []
    logger.info(f"Starting to process {len(links)} items")

    for idx, link in enumerate(links, 1):
        logger.info(f"[{idx}/{len(links)}] Processing: {link}")
        data = extract_item_data(link)
        if data:
            results.append(data)
            logger.debug(f"Added item {data.get('item_id')} to results")
        else:
            logger.warning(f"Failed to process item: {link}")

    # Save to JSON
    try:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(results)} items to {OUTPUT_FILE}")
        if len(results) != len(links):
            logger.warning(f"Processed {len(results)} out of {len(links)} items ({(len(results)/len(links))*100:.1f}% success rate)")
    except Exception as e:
        logger.error(f"Failed to save data to {OUTPUT_FILE}: {str(e)}", exc_info=True)


# Run the scraper
scrape_all_items()

2025-05-12 00:30:34 - scraper - INFO - Starting to process 7364 items
2025-05-12 00:30:34 - scraper - INFO - [1/7364] Processing: 5165411
2025-05-12 00:30:35 - scraper - INFO - Successfully processed item: 5165411
2025-05-12 00:30:35 - scraper - INFO - [2/7364] Processing: 5150136
2025-05-12 00:30:35 - scraper - INFO - Successfully processed item: 5150136
2025-05-12 00:30:35 - scraper - INFO - [3/7364] Processing: 5117823
2025-05-12 00:30:36 - scraper - INFO - Successfully processed item: 5117823
2025-05-12 00:30:36 - scraper - INFO - [4/7364] Processing: 5193713
2025-05-12 00:30:37 - scraper - INFO - Successfully processed item: 5193713
2025-05-12 00:30:37 - scraper - INFO - [5/7364] Processing: 5193372
2025-05-12 00:30:38 - scraper - INFO - Successfully processed item: 5193372
2025-05-12 00:30:38 - scraper - INFO - [6/7364] Processing: 4286568
2025-05-12 00:30:39 - scraper - INFO - Successfully processed item: 4286568
2025-05-12 00:30:39 - scraper - INFO - [7/7364] Processing: 519410